# Загрузка данных анкет и проведение МАИ, анализ полученных результатов
Данные: заполненные анкеты экспертов (таблицы Excel) 

In [2]:
import re
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from random import randint

In [3]:
df = pd.read_excel(r'C:\Users\user\предпосылки\Экс7.xlsx')
df.head()

,Тип нарушения,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала
0,Документация,1.00,0.25,0.2,4.0,0.25,6.000000,6.00
1,Экспертиза промышленной безопасности,4.00,1.00,0.2,0.5,4.00,0.142857,0.25
2,Готовность к ликвидации последствий аварий и и...,5.00,5.00,1.0,5.0,0.25,0.200000,4.00
3,Оформление информационными знаками и знаками б...,0.25,2.00,0.2,1.0,0.25,0.142857,0.25
4,Содержание объектов и оборудования,4.00,0.25,4.0,4.0,1.00,0.142857,0.25


In [4]:
print('Число строк и столбцов в наборе данных:\n', df.shape)

Число строк и столбцов в наборе данных:
 (7, 8)


In [5]:
# типы столбцов 
df.dtypes

Тип нарушения                                                 object
Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов      float64
Оформление информационными знаками и знаками безопасности    float64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала       float64
dtype: object

In [6]:
#удаление столбца типа object для дальнейшей работы с фреймом 
df.pop('Тип нарушения') 
print(df)

   Документация   Экспертиза промышленной безопасности  \
0       1.000000                                  0.25   
1       4.000000                                  1.00   
2       5.000000                                  5.00   
3       0.250000                                  2.00   
4       4.000000                                  0.25   
5       0.166667                                  7.00   
6       0.166667                                  4.00   

   Готовность к ликвидации последствий аварий и инцидентов  \
0                                               0.20         
1                                               0.20         
2                                               1.00         
3                                               0.20         
4                                               4.00         
5                                               5.00         
6                                               0.25         

   Оформление информационными знаками 

In [7]:
# типы столбцов 
df.dtypes

Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов      float64
Оформление информационными знаками и знаками безопасности    float64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала       float64
dtype: object

In [8]:
# также меням тип данных Документации
df['Документация '] = df['Документация '].astype(float)

In [9]:
df.loc[:, df.columns[df.dtypes == 'float64']].head(7)

,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала
0,1.000000,0.25,0.20,4.0,0.25,6.000000,6.00
1,4.000000,1.00,0.20,0.5,4.00,0.142857,0.25
2,5.000000,5.00,1.00,5.0,0.25,0.200000,4.00
3,0.250000,2.00,0.20,1.0,0.25,0.142857,0.25
4,4.000000,0.25,4.00,4.0,1.00,0.142857,0.25
5,0.166667,7.00,5.00,7.0,7.00,1.000000,7.00
6,0.166667,4.00,0.25,4.0,4.00,0.142857,1.00


In [10]:
DF_1=df[:].copy()

In [11]:
#Найдем сумму всех столбцов, для того чтобы в дальнейшем пересчитать полученные значения
df.sum ()

Документация                                                 14.583333
Экспертиза промышленной безопасности                         19.500000
Готовность к ликвидации последствий аварий и инцидентов      10.850000
Оформление информационными знаками и знаками безопасности    25.500000
Содержание объектов и оборудования                           16.750000
Охранные зоны (ОЗ)                                            7.771429
Организация рабочего процесса и безопасности персонала       18.750000
dtype: float64

In [12]:
# Считаем значения для нашей матрицы, путем деления каждого значения в столбце на сумму столбца
df['new_Документация'] = df['Документация '].astype(float).div(df['Документация '].sum())
df['new_Экспертиза'] = df['Экспертиза промышленной безопасности'].astype(float).div(df['Экспертиза промышленной безопасности'].sum())
df['new_Готовность'] = df['Готовность к ликвидации последствий аварий и инцидентов'].astype(float).div(df['Готовность к ликвидации последствий аварий и инцидентов'].sum())
df['new_Оформление'] = df['Оформление информационными знаками и знаками безопасности'].astype(float).div(df['Оформление информационными знаками и знаками безопасности'].sum())
df['new_Содержание'] = df['Содержание объектов и оборудования '].astype(float).div(df['Содержание объектов и оборудования '].sum())
df['new_ОЗ'] = df['Охранные зоны (ОЗ)'].astype(float).div(df['Охранные зоны (ОЗ)'].sum())
df['new_Организация'] = df['Организация рабочего процесса и безопасности персонала'].astype(float).div(df['Организация рабочего процесса и безопасности персонала'].sum())

In [13]:
df.head()

,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,1.00,0.25,0.2,4.0,0.25,6.000000,6.00,0.068571,0.012821,0.018433,0.156863,0.014925,0.772059,0.320000
1,4.00,1.00,0.2,0.5,4.00,0.142857,0.25,0.274286,0.051282,0.018433,0.019608,0.238806,0.018382,0.013333
2,5.00,5.00,1.0,5.0,0.25,0.200000,4.00,0.342857,0.256410,0.092166,0.196078,0.014925,0.025735,0.213333
3,0.25,2.00,0.2,1.0,0.25,0.142857,0.25,0.017143,0.102564,0.018433,0.039216,0.014925,0.018382,0.013333
4,4.00,0.25,4.0,4.0,1.00,0.142857,0.25,0.274286,0.012821,0.368664,0.156863,0.059701,0.018382,0.013333


In [14]:
# Получили новую таблицу теперь отделим значения нашей матрицы от основной таблицы
df_new = df[['new_Документация', 'new_Экспертиза', 'new_Готовность', 'new_Оформление', 'new_Содержание', 'new_ОЗ', 'new_Организация']]
df_new

,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,0.068571,0.012821,0.018433,0.156863,0.014925,0.772059,0.320000
1,0.274286,0.051282,0.018433,0.019608,0.238806,0.018382,0.013333
2,0.342857,0.256410,0.092166,0.196078,0.014925,0.025735,0.213333
3,0.017143,0.102564,0.018433,0.039216,0.014925,0.018382,0.013333
4,0.274286,0.012821,0.368664,0.156863,0.059701,0.018382,0.013333
5,0.011429,0.358974,0.460829,0.274510,0.417910,0.128676,0.373333
6,0.011429,0.205128,0.023041,0.156863,0.238806,0.018382,0.053333


In [15]:
#Вычислим средние значение по строке и запишем полученный результот в mean
df_new['mean'] = df_new.mean(axis=1)
df_new.head()

C:\Users\user\anaconda3\envs\envpy3.5\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация,mean
0,0.068571,0.012821,0.018433,0.156863,0.014925,0.772059,0.320000,0.194810
1,0.274286,0.051282,0.018433,0.019608,0.238806,0.018382,0.013333,0.090590
2,0.342857,0.256410,0.092166,0.196078,0.014925,0.025735,0.213333,0.163072
3,0.017143,0.102564,0.018433,0.039216,0.014925,0.018382,0.013333,0.032000
4,0.274286,0.012821,0.368664,0.156863,0.059701,0.018382,0.013333,0.129150


In [16]:
#проверка, среднее значение в сумме должно быть равно 1
df_new['mean'].sum()

1.0

In [17]:
# Далее проведем проверку на согласованность, 
#для этого создадим из нашей полученной выше таблицы матрицы для дальнейшего их перемножения
df_new8 =df_new['mean']

# удаляем из нашей таблицы значениея столбцы mean
#df_new.pop('mean')

df_new1 =DF_1.loc[0]

df_new2 =DF_1.loc[1]

df_new3 =DF_1.loc[2]

df_new4 =DF_1.loc[3]

df_new5 =DF_1.loc[4]

df_new6 =DF_1.loc[5]

df_new7 =DF_1.loc[6]

In [18]:
#Проверка на согласованность
res1 = np.dot(df_new1,df_new8)
print(res1)
res2 = np.dot(df_new2,df_new8)
print(res2)
res3 = np.dot(df_new3,df_new8)
print(res3)
res4 = np.dot(df_new4,df_new8)
print(res4)
res5 = np.dot(df_new5,df_new8)
print(res5)
res6 = np.dot(df_new6,df_new8)
print(res6)
res7 = np.dot(df_new7,df_new8)
print(res7)

2.7526252268832705
1.5016347548637556
2.24422546245303
0.39337362717038715
1.777915297890361
3.606369700462375
1.2225323455682755


In [19]:
sum_new = res1 + res2 + res3 + res4 + res5 + res6 + res7
print(sum_new)

13.498676415291454


In [20]:
# Вычислем значения CI, RI, CR
CI = (sum_new - 7)/(7-1)
print(CI)
RI = (1.98*(7-2))/7
print(RI)
CR = CI/RI
print(CR)

1.083112735881909
1.4142857142857144
0.7658372879973093


In [21]:
# Получилось, что значение CR=0.76, что большее 0,1 значит матрицу нельзя считать согласованной
# Проделаем аналогичные действия с другими экспертами